### Zadanie 1. - Przygotowanie zbioru

Przygotuj wybrany zestaw danych w formacie ARFF (jeśli to możliwe, użyj zestawów danych z poprzedniego
laboratorium)

In [1]:
import pandas as pd

def csv_to_arff(csv_filename, arff_filename, relation_name='water_dataset'):
    # 1. Wczytanie danych
    try:
        df = pd.read_csv(csv_filename)
        print(f"Pomyślnie wczytano plik: {csv_filename}")
    except FileNotFoundError:
        print("Błąd: Nie znaleziono pliku CSV. Sprawdź ścieżkę.")
        return

    # Otwieramy plik do zapisu
    with open(arff_filename, 'w', encoding='utf-8') as f:

        # 2. Nagłówek @RELATION
        f.write(f"@RELATION {relation_name}\n\n")

        # 3. Definicja atrybutów (@ATTRIBUTE)
        # Dla Weka najlepiej zdefiniować te wartości jako nominalne {opcja1, opcja2}
        for column in df.columns:
            # Pobieramy unikalne wartości z kolumny, sortujemy i zamieniamy na string
            unique_values = sorted(df[column].unique())

            # Tworzymy listę wartości w formacie {val1, val2, val3}
            # np. {range1, range2} lub {0, 1}
            values_str = ",".join([str(v) for v in unique_values])

            f.write(f"@ATTRIBUTE {column} {{{values_str}}}\n")

        # 4. Sekcja danych (@DATA)
        f.write("\n@DATA\n")

        # Zapisujemy każdy wiersz
        for index, row in df.iterrows():
            # Łączymy wartości przecinkami
            row_str = ",".join([str(v) for v in row.values])
            f.write(f"{row_str}\n")

    print(f"Gotowe! Plik zapisano jako: {arff_filename}")
    print("Możesz teraz otworzyć ten plik w programie Weka.")

# --- URUCHOMIENIE ---
input_csv = 'WATER_POTABILITY_CLEANED.csv'
output_arff = 'water_potability.arff'

csv_to_arff(input_csv, output_arff)

Pomyślnie wczytano plik: WATER_POTABILITY_CLEANED.csv
Gotowe! Plik zapisano jako: water_potability.arff
Możesz teraz otworzyć ten plik w programie Weka.


### Zadanie 2. - Generowanie rankingów

Wygeneruj 2 rankingi używając 2 różnych metod generowania rankingów.
Opisz szczegółowo, jakie parametry zostały użyte do wygenerowania rankingów:
Uzasadnij, dlaczego wybrane parametry zostały użyte.


=== Run information ===

Evaluator:    weka.attributeSelection.InfoGainAttributeEval
Search:       weka.attributeSelection.Ranker -T -1.7976931348623157E308 -N -1
Relation:     water_dataset
Instances:    419
Attributes:   10
              ph
              Hardness
              Solids
              Chloramines
              Sulfate
              Conductivity
              Organic_carbon
              Trihalomethanes
              Turbidity
              Potability
Evaluation mode:    evaluate on all training data



=== Attribute Selection on all input data ===

Search Method:
	Attribute ranking.

Attribute Evaluator (supervised, Class (nominal): 10 Potability):
	Information Gain Ranking Filter

Ranked attributes:
 0.030031   3 Solids
 0.024472   5 Sulfate
 0.010397   2 Hardness
 0.006482   7 Organic_carbon
 0.004139   1 ph
 0.003751   4 Chloramines
 0.001238   6 Conductivity
 0.000354   9 Turbidity
 0.000103   8 Trihalomethanes

Selected attributes: 3,5,2,7,1,4,6,9,8 : 9



=== Run information ===

Evaluator:    weka.attributeSelection.CorrelationAttributeEval
Search:       weka.attributeSelection.Ranker -T -1.7976931348623157E308 -N -1
Relation:     water_dataset
Instances:    419
Attributes:   10
              ph
              Hardness
              Solids
              Chloramines
              Sulfate
              Conductivity
              Organic_carbon
              Trihalomethanes
              Turbidity
              Potability
Evaluation mode:    evaluate on all training data



=== Attribute Selection on all input data ===

Search Method:
	Attribute ranking.

Attribute Evaluator (supervised, Class (nominal): 10 Potability):
	Correlation Ranking Filter
Ranked attributes:
 0.204    3 Solids
 0.1842   5 Sulfate
 0.12     2 Hardness
 0.0946   7 Organic_carbon
 0.0757   1 ph
 0.0721   4 Chloramines
 0.0414   6 Conductivity
 0.0222   9 Turbidity
 0.012    8 Trihalomethanes

Selected attributes: 3,5,2,7,1,4,6,9,8 : 9



### Metoda 1: Information Gain (Zysk Informacyjny)
* **Algorytm w Weka:** `InfoGainAttributeEval` + `Ranker`
* **Parametry:** Ustawienia domyślne (binarizeNumericAttributes=false, missingMerge=true).
* **Uzasadnienie:** Wybrano tę metodę, ponieważ mierzy ona redukcję entropii (nieuporządkowania) klasy decyzyjnej po podziale zbioru względem danego atrybutu. Jest to standardowa miara stosowana w drzewach decyzyjnych, idealna dla danych kategorialnych, pozwalająca wskazać cechy niosące najwięcej "czystej" informacji.

### Metoda 2: Correlation (Korelacja)
* **Algorytm w Weka:** `CorrelationAttributeEval` + `Ranker`
* **Parametry:** Ustawienia domyślne.
* **Uzasadnienie:** Metoda ta ocenia wartość atrybutu poprzez obliczenie korelacji (zależności) między nim a klasą decyzyjną. Pozwala to na szybką identyfikację cech, które wykazują najsilniejszy związek z wynikiem końcowym (Potability). Jest to jedna z najczęściej stosowanych metod filtrowania cech, która pomaga wyeliminować atrybuty mało znaczące lub niezwiązane z badanym problemem.

### Parametry Rankera:
* generateRanking = true – generowanie pełnej listy rankingowej cech,
* numToSelect = -1 – brak ograniczenia liczby wybieranych atrybutów (uwzględniono wszystkie cechy),
* startSet = ∅ – brak atrybutów początkowych,
* threshold = -1.7976931348623157E308 – brak progu odrzucania cech (najmniejsza możliwa wartość typu double).

**Wyniki rankingów (Top 4 cechy):**
Obie metody wskazały tę samą czołówkę atrybutów, co świadczy o wysokiej stabilności znaczenia tych cech w zbiorze:
1. **Solids**
2. **Sulfate**
3. **Hardness**
4. **Organic_carbon**

### Zadanie 3. - Klasyfikacja

Porównaj skuteczność modeli klasyfikacyjnych (np. drzewa decyzyjne) przy użyciu pełnego zbioru cech i
zestawu zredukowanego na podstawie rankingu.
Porównanie to wykonaj dla obu wygenerowanych rankingów.


=== Run information ===

Scheme:       weka.classifiers.trees.J48 -C 0.25 -M 2
Relation:     water_dataset
Instances:    419
Attributes:   10
              ph
              Hardness
              Solids
              Chloramines
              Sulfate
              Conductivity
              Organic_carbon
              Trihalomethanes
              Turbidity
              Potability
Test mode:    10-fold cross-validation

=== Classifier model (full training set) ===

J48 pruned tree
------------------

Solids = range1
|   Sulfate = range1
|   |   Hardness = range1
|   |   |   ph = range1: 0 (26.0/3.0)
|   |   |   ph = range2
|   |   |   |   Trihalomethanes = range1: 1 (8.0/2.0)
|   |   |   |   Trihalomethanes = range2: 0 (14.0/4.0)
|   |   Hardness = range2
|   |   |   Organic_carbon = range1: 1 (32.0/12.0)
|   |   |   Organic_carbon = range2: 0 (29.0/10.0)
|   Sulfate = range2: 0 (128.0/28.0)
Solids = range2
|   Hardness = range1
|   |   Sulfate = range1: 1 (32.0/6.0)
|   |   Sulfate = range2
|   |   |   Turbidity = range1
|   |   |   |   Trihalomethanes = range1: 1 (11.0/1.0)
|   |   |   |   Trihalomethanes = range2
|   |   |   |   |   Organic_carbon = range1: 1 (6.0/1.0)
|   |   |   |   |   Organic_carbon = range2: 0 (7.0/2.0)
|   |   |   Turbidity = range2
|   |   |   |   ph = range1: 0 (13.0/4.0)
|   |   |   |   ph = range2
|   |   |   |   |   Organic_carbon = range1: 0 (7.0/2.0)
|   |   |   |   |   Organic_carbon = range2: 1 (8.0)
|   Hardness = range2
|   |   Sulfate = range1
|   |   |   ph = range1
|   |   |   |   Organic_carbon = range1
|   |   |   |   |   Trihalomethanes = range1
|   |   |   |   |   |   Conductivity = range1: 1 (3.0/1.0)
|   |   |   |   |   |   Conductivity = range2: 0 (2.0)
|   |   |   |   |   Trihalomethanes = range2
|   |   |   |   |   |   Chloramines = range1: 0 (3.0/1.0)
|   |   |   |   |   |   Chloramines = range2: 1 (3.0)
|   |   |   |   Organic_carbon = range2: 0 (7.0)
|   |   |   ph = range2
|   |   |   |   Chloramines = range1
|   |   |   |   |   Organic_carbon = range1
|   |   |   |   |   |   Trihalomethanes = range1: 1 (3.0/1.0)
|   |   |   |   |   |   Trihalomethanes = range2: 0 (3.0)
|   |   |   |   |   Organic_carbon = range2: 1 (3.0/1.0)
|   |   |   |   Chloramines = range2: 1 (14.0/4.0)
|   |   Sulfate = range2
|   |   |   Chloramines = range1
|   |   |   |   Trihalomethanes = range1: 0 (14.0/3.0)
|   |   |   |   Trihalomethanes = range2
|   |   |   |   |   Turbidity = range1: 0 (7.0/1.0)
|   |   |   |   |   Turbidity = range2: 1 (6.0/1.0)
|   |   |   Chloramines = range2: 0 (30.0/5.0)

Number of Leaves  : 	26

Size of the tree : 	51


Time taken to build model: 0.02 seconds

=== Stratified cross-validation ===
=== Summary ===

Correctly Classified Instances         271               64.6778 %
Incorrectly Classified Instances       148               35.3222 %
Kappa statistic                          0.2289
Mean absolute error                      0.405
Root mean squared error                  0.5157
Relative absolute error                 85.362  %
Root relative squared error            105.8986 %
Total Number of Instances              419

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,770    0,549    0,690      0,770    0,728      0,232    0,628     0,697     0
                 0,451    0,230    0,553      0,451    0,497      0,232    0,628     0,490     1
Weighted Avg.    0,647    0,426    0,637      0,647    0,638      0,232    0,628     0,617

=== Confusion Matrix ===

   a   b   <-- classified as
 198  59 |   a = 0
  89  73 |   b = 1



=== Run information ===

Scheme:       weka.classifiers.trees.J48 -C 0.25 -M 2
Relation:     water_dataset-weka.filters.unsupervised.attribute.Remove-R1,4,6,8-9
Instances:    419
Attributes:   5
              Hardness
              Solids
              Sulfate
              Organic_carbon
              Potability
Test mode:    10-fold cross-validation

=== Classifier model (full training set) ===

J48 pruned tree
------------------

Solids = range1
|   Sulfate = range1
|   |   Hardness = range1: 0 (48.0/13.0)
|   |   Hardness = range2
|   |   |   Organic_carbon = range1: 1 (32.0/12.0)
|   |   |   Organic_carbon = range2: 0 (29.0/10.0)
|   Sulfate = range2: 0 (128.0/28.0)
Solids = range2
|   Hardness = range1: 1 (84.0/27.0)
|   Hardness = range2: 0 (98.0/34.0)

Number of Leaves  : 	6

Size of the tree : 	11


Time taken to build model: 0 seconds

=== Stratified cross-validation ===
=== Summary ===

Correctly Classified Instances         285               68.0191 %
Incorrectly Classified Instances       134               31.9809 %
Kappa statistic                          0.2832
Mean absolute error                      0.4258
Root mean squared error                  0.4694
Relative absolute error                 89.7429 %
Root relative squared error             96.3852 %
Total Number of Instances              419

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,840    0,574    0,699      0,840    0,763      0,295    0,628     0,684     0
                 0,426    0,160    0,627      0,426    0,507      0,295    0,628     0,521     1
Weighted Avg.    0,680    0,414    0,671      0,680    0,664      0,295    0,628     0,621

=== Confusion Matrix ===

   a   b   <-- classified as
 216  41 |   a = 0
  93  69 |   b = 1



Przeprowadzono porównanie skuteczności modelu klasyfikacyjnego na pełnym oraz zredukowanym zbiorze cech.

* **Klasyfikator:** J48 (implementacja algorytmu C4.5)
* **Metoda walidacji:** 10-krotna walidacja krzyżowa (Cross-validation 10 folds)

### Wyniki eksperymentu:

| Zbiór danych | Liczba atrybutów | Dokładność (Accuracy) | Szczegóły redukcji |
| :--- | :--- |:----------------------| :--- |
| **Pełny zbiór** | 9 | **64.68%**            | Wszystkie dostępne cechy. |
| **Zredukowany** | 4 | **68.02%**            | Pozostawiono tylko: *Solids, Sulfate, Hardness, Organic_carbon*. |

**Obserwacja:**
Usunięcie 5 najsłabiej ocenionych atrybutów (według rankingów z Zadania 2) spowodowało **wzrost** skuteczności klasyfikacji o  3.34 punktów procentowych.

Z uwagi na fakt, że obie metody rankingowe wyłoniły identyczny zestaw 4 najważniejszych cech, wynik klasyfikacji dla zbioru zredukowanego jest wspólny dla obu metod selekcji

### Zadanie 4. - Analiza wyników

Oceń stabilność rankingów - Czy różne algorytmy generują podobne rankingi cech dla tego samego zbioru
danych.
Które cechy są najważniejsze w kontekście wybranego zbioru danych

### 1. Ocena stabilności rankingów
Porównując wyniki uzyskane metodami *Information Gain* oraz *Correlation*, stwierdzono **wysoką stabilność rankingów**.
Obie metody wygenerowały identyczną kolejność dla czterech najważniejszych atrybutów (*Solids, Sulfate, Hardness, Organic_carbon*). Wskazuje to, że w badanym zbiorze danych najważniejsze zależności są wyraźne i nie zależą od specyfiki wybranego algorytmu selekcji.

### 2. Najważniejsze cechy
W kontekście badanego problemu (zdatność wody do picia), kluczowymi parametrami okazały się:
* **Solids (Substancje stałe):** Lider obu rankingów.
* **Sulfate (Siarczany) i Hardness (Twardość):** Kolejne istotne zmienne.

Cechy takie jak *Turbidity* (Mętność) czy *Trihalomethanes* znalazły się na końcu rankingów, co sugeruje, że w tym konkretnym, przetworzonym zbiorze danych nie mają one statystycznie istotnego wpływu na klasę decyzyjną.

### 3. Wnioski końcowe
Eksperyment potwierdził użyteczność selekcji cech w procesie odkrywania wiedzy.
* Redukcja wymiarowości pozwoliła na usunięcie szumu informacyjnego.
* Model zbudowany na mniejszej liczbie cech okazał się nie tylko prostszy, ale i znacznie skuteczniejszy (wzrost accuracy z ~64% do ~68%).
* Wskazuje to, że w analizowanym zbiorze występuje wiele atrybutów nadmiarowych, które zamiast pomagać, utrudniały działanie algorytmu J48 na pełnym zbiorze danych.